In [43]:
import ipywidgets as ipw
import np_session 
import IPython 
import enum

class Workflow(enum.Enum):
    TEST = "test"
    EPHYS = "ephys"
    
class Experiment:
    task_name: str = ""
    preset_task_names = ("test1", "ephys aud","ephys vis")
    mouse = np_session.Mouse(0)
    
def task_select_widget(
    experiment,
) -> None:
    """Select a task name for controlling behavior of TaskControl.
    """
    task_dropdown = ipw.Select(
        options=tuple(set(task for task in experiment.preset_task_names)),
        description="Presets",
        layout=ipw.Layout(min_width="400px", max_height="400px"),
    )
    task_input_box = ipw.Text(
        value=experiment.task_name if isinstance(experiment.task_name, str) else "",
        continuous_update=False,
    )
    console = ipw.Output()
    with console:
        if last_task:= experiment.mouse.state.get('last_task'):
            print(f"{experiment.mouse} last task: {last_task}")

    def update(change):
        if change["name"] != "value":
            return
        if (options := getattr(change["owner"], "options", None)) and change[
            "new"
        ] not in options:
            return
        if change["new"] == change["old"]:
            return
        print(change)
        if change["owner"] is task_dropdown:
            experiment.task_name = str(task_dropdown.value)
            task_input_box.value = experiment.task_name
            return
        elif change["owner"] is task_input_box:
            experiment.task_name = str(task_input_box.value)
            if str(task_dropdown.value) != experiment.task_name:
                task_dropdown.value = None
        with console:
            print(f"Updated task: {experiment.task_name}")
    task_dropdown.observe(update, names='value')
    task_input_box.observe(update, names='value')

    IPython.display.display(ipw.VBox([ipw.HBox([task_dropdown, task_input_box]), console]))
task_select_widget(Experiment())

In [1]:
%env AIBS_RIG_ID=NP1
# for testing

env: AIBS_RIG_ID=NP1


In [2]:
%load_ext autoreload
%autoreload 2
import np_workflows 
import IPython.display
import np_services
np_workflows.quiet_mode_widget()

ToggleButton(value=True, button_style='info', description='Quiet mode is on', icon='check', tooltip='Quiet mod…

In [3]:
import np_workflows.experiments.templeton as Templeton
user, mouse = np_workflows.user_and_mouse_widget()

In [5]:
from np_workflows.experiments.templeton.templeton_widgets import workflow_select_widget
workflow_select_widget(mouse)

TempletonSelectedWorkflow(TempletonWorkflow.PRETEST, 366122)

In [8]:
import np_workflows.experiments.templeton as Templeton
import pathlib, IPython.display, np_probe_targets, contextlib

COMBO_1 = np_probe_targets.ProbeInsertionsTempleton({
    'A':'A1',
    'B':'B1', # or B3
    'C':'C2',
    'D': None,
    'E': None,
    'F':'F1',
    }
)
COMBO_2 = np_probe_targets.ProbeInsertionsTempleton({
    'A':'A2', # or A3
    'B':'B2',
    'C':'C1', # or C4
    'D': 'D1',
    'E': None,
    'F':'F2',
    }
)
    
targets = COMBO_2

targets.save_dir = pathlib.Path('//allen/programs/mindscope/workgroups/dynamicrouting/ben')
with contextlib.suppress(Exception):
    targets.save_dir = experiment.npexp_path
    
    
IPython.display.display(
    np_probe_targets.ProbeTargetInsertionRecordWidget(
        targets, 
        implant_drawing=np_probe_targets.TempletonDrawingSVGProbeColormap, 
        # implant_drawing=np_probe_targets.TempletonDrawingSVGComboColormap, 
        current_insertion_group=np_probe_targets.ProbeInsertionsTempleton)
    )

ProbeTargetInsertionRecordWidget(children=(Output(), VBox(children=(HBox(children=(VBox(children=(SelectionSli…

In [ ]:

user, mouse = np_workflows.user_and_mouse_widget()

In [ ]:
np_workflows.mtrain_widget(668800)

In [ ]:
import np_session, np_tools

for session in np_session.sessions():
    if not isinstance(session, np_session.PipelineSession):
        continue
    if not session.lims_path:
        continue
    for probes in (f'{session.folder}_probe{suffix}' for suffix in ('ABC', 'DEF')):
        if (
            npexp := (session.npexp_path / probes)
        ).exists() and (
            lims := (session.lims_path / probes)
        ).exists():
            for data in npexp.rglob('*continuous.dat'):
                lims_copy = lims / data.relative_to(npexp)
                sizes_match = data.stat().st_size == lims_copy.stat().st_size
                if not sizes_match:
                    print(f'{npexp} sizes do not match')
                    continue
                if np_tools.checksums_match(data, lims / data.relative_to(npexp)):
                    data.unlink()

In [ ]:
import np_session
session = np_session.Session(1263794343)

In [ ]:
np_workflows.mtrain_widget('668800')

In [ ]:
experiment.session = session

In [ ]:
for _ in experiment.services:
    try:
        print(_.get_latest_data())
    except:
        pass

In [ ]:
experiment = TTN.Ephys(mouse, user)

In [ ]:
import np_workflows.experiments.task_trained_network as TTN


In [ ]:
np_services.Sync.get_latest_data()

In [ ]:
print(dye_info_widget.__doc__)

In [ ]:
import contextlib
from np_workflows import npxc 
import np_session
import ipywidgets as ipw

def dye_info_widget(session: np_session.Session) -> IPython.display.DisplayHandle | None:
    """`di_widget`, with dye entries in a database.
    
    - scan barcode or enter ID number for the dye used
    - change dye description if incorrect (DiI, DiO)
    - increment number of times probes were dipped this session
    - hit `Save` to store info in platform.json
    """
    
    di_info: dict[str, int | str] = dict(
        EndTime=0, StartTime=npxc.now(), dii_description="", times_dipped=0, previous_uses="",
    )
    di_info.update(session.platform_json.DiINotes)
    
    width = lambda w: ipw.Layout(max_width=f'{w}px')
    
    dye_id_entry = ipw.Text(value=None, description='Dye ID', layout=width(250), placeholder='Enter ID or scan barcode')
    dye_usage_button = ipw.Button(description='Record single use', button_style='warning', layout=width(180))
    first_usage = ipw.Text(value='', description="First use", layout=width(250), disabled=True)
    dye_dropdown = ipw.Dropdown(description="Description:", options=np_session.Dye.descriptions, layout=width(180))
    dipped_counter = ipw.IntText(value=di_info['times_dipped'], min=0, max=99, description="Dipped count", layout=width(150))
    usage_counter = ipw.IntText(value=int(di_info['previous_uses']), min=0, max=99, description="Previous uses", layout=width(180), disabled=True)
    save_button = ipw.Button(description='Save', button_style='warning', layout=width(180))
    if (desc := di_info['dii_description']) in np_session.Dye.descriptions:
        dye_dropdown.value = desc
        
    def update_display(_):
        with contextlib.suppress(Exception):
            dye = np_session.Dye(int(str(dye_id_entry.value)))
            dye_dropdown.value = dye.description
            usage_counter.value = dye.previous_uses
            first_usage.value = f'{dye.first_use}'
    dye_id_entry.observe(update_display, 'value')
    
    def record_dye_usage():
        with contextlib.suppress(Exception):
            dye = np_session.Dye(int(str(dye_id_entry.value)))
            dye.description = dye_dropdown.value
            dye.increment_uses()
        
    def update_di_info():
        di_info['EndTime'] = npxc.now()
        di_info['times_dipped'] = str(dipped_counter.value)
        di_info['dii_description'] = str(dye_dropdown.value)
        di_info['previous_uses'] = str(usage_counter.value)
        
    def on_click(b):
        update_di_info()
        record_dye_usage()
        session.platform_json.DiINotes = di_info
        save_button.description = 'Saved'
        save_button.button_style = 'success'
        
    save_button.on_click(on_click)
    return IPython.display.display(ipw.VBox([
        dye_id_entry,
        dipped_counter, dye_dropdown, 
        usage_counter, first_usage, save_button]))

import IPython.display
# IPython.display.display(
dye_info_widget(np_session.Session('1246096278_366122_20230209'))
# np_workflows.di_widget(np_session.Session('1246096278_366122_20230209'))
# )   


In [ ]:
import np_session
mouse = np_session.LIMS2MouseInfo(661728)
exp_id = mouse.isi_id
exps = mouse.isi_info['isi_experiments']
isi = [e for e in exps if e['id'] == exp_id]

In [ ]:
mouse.isi_id

In [ ]:
from np_workflows import npxc 
import ipywidgets as ipw

def di_widget(session: np_session.Session) -> IPython.display.DisplayHandle | None:
    "Supply a path or a platform json instance. Saves a JSON file with the dye used in the session and a timestamp."

    di_info: dict[str, int | str] = dict(
        EndTime=0, StartTime=npxc.now(), dii_description="", times_dipped=0, previous_uses="",
    )
    di_info.update(session.platform_json.DiINotes)
    
    layout = ipw.Layout(max_width='180px')
    dipped_counter = ipw.IntText(value=0, min=0, max=99, description="Dipped count", layout=layout)
    usage_counter = ipw.IntText(value=session, min=0, max=99, description="Previous uses", layout=layout)
    dye_dropdown = ipw.Dropdown(options=['CM-DiI 100%', 'DiO'], layout=layout)
    save_button = ipw.Button(description='Save', button_style='warning', layout=layout)
    
    def update_di_info():
        di_info['EndTime'] = npxc.now()
        di_info['times_dipped'] = str(dipped_counter.value)
        di_info['dii_description'] = str(dye_dropdown.value)
        di_info['previous_uses'] = str(usage_counter.value)
        
    def on_click(b):
        update_di_info()
        session.platform_json.DiINotes = di_info
        save_button.description = 'Saved'
        save_button.button_style = 'success'
        
    save_button.on_click(on_click)
    return IPython.display.display(ipw.VBox([
        dipped_counter, dye_dropdown, 
        usage_counter, save_button]))

import IPython.display
IPython.display.display(
    di_widget(np_session.Session('1246096278_366122_20230209'))
)

In [ ]:
np_workflows.isi_widget(661728)

In [ ]:
np_workflows.restart_kernel()
from ipylab import JupyterFrontEnd

app = JupyterFrontEnd()
app.commands.execute('kernelmenu:restart')
np_services

In [ ]:
import np_workflows 
import IPython.display
import np_services
_ = np_workflows.npxc.now()
np_services.normalize_time(_)

In [ ]:
import np_logging
logger =np_logging.getLogger()
import np_config

In [ ]:
folders = np_services.OpenEphys.data_files
folders = [np_config.local_to_unc(np_config.Rig().acq, f'{_}:/1247605185_366122_20230215') for _ in 'AB']
if not folders:
    logger.info('Renaming: no ephys folders have been recorded')
for name in set(_.name for _ in folders):
    if length := len(split_folders := [_ for _ in folders if _.name == name]) != 2:
        logger.info(f'Renaming: {length} folders found for {name}, expected 2 - aborted')
        # return
    logger.debug('Renaming split ephys folders %r', split_folders)
    for folder, probe_letters in zip(sorted(split_folders, key=lambda x: x.as_posix()), ('ABC', 'DEF')):
        folder.replace(folder.with_name(f'{name}_probe{probe_letters}'))
    logger.debug('Renaming split ephys folders %r', split_folders)


In [ ]:
np_workflows.quiet_mode_widget()

In [ ]:
import np_logging
np_logging.get_logger().info(' ')
np_logging.get_logger().debug(' ')
1/0

In [ ]:
user, mouse = np_workflows.user_and_mouse_widget()

In [ ]:
#! for testing
import np_session, np_services

session = np_session.Session('1246096278_366122_20230209')
session_folder = session.npexp_path
platform_json = session.platform_json
session.platform_json.path
session.platform_json.mouseID = 366122
print(session.platform_json.path)
IPython.display.JSON(session.platform_json.path)

In [ ]:
import np_session
session = np_session.Session('1246096278_366122_20230209')


In [ ]:
%%timeit
_ = np_config.from_zk('logins')

In [ ]:
%%timeit
session.state['test'] = session.platform_json.json()

In [ ]:
%%timeit
session.state['test'] = 1

In [ ]:
%%timeit
_ = session.state['test']

In [ ]:
%%timeit
_ = session.platform_json.path.stat().st_mtime

In [ ]:
%%timeit
session.platform_json.load_from_existing()

In [ ]:
%%timeit
session.platform_json.path.read_bytes()

In [ ]:
%%timeit
session.platform_json.path.read_text()

In [ ]:
def insertion_notes_widget(session: np_session.Session):
    
    probes = 'ABCDEF'
    probe = lambda _: f'Probe{_}'
    fields = (
        "FailedToInsert",
        # "ProbeLocationChanged",
        # "ProbeBendingOnSurface",
        # "ProbeBendingElsewhere",
    )
    # "NumAgarInsertions",
    
    get_notes = lambda _: session.platform_json.InsertionNotes.get(probe(_), {}).get('Notes', '')
    get_field = lambda _, field: session.platform_json.InsertionNotes.get(probe(_), {}).get(field, None)
    
    def disp_str(s): # split PascalCase fieldname into 'Title case' words
        matches = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', s)
        return ' '.join([m.group(0) for m in matches]).lower().capitalize()
    save_str = lambda s: ''.join([_.capitalize() for _ in s.split(' ')])
    
    row = lambda *args: ipw.HBox([*args])
    probe_row = lambda p: row(
        ipw.Text(value=get_notes(p), placeholder='Insertion notes', description=disp_str(probe(p).strip('Probe ')), layout=ipw.Layout(width='auto', min_width='400px')),
        *(ipw.Checkbox(value=get_field(p, field), description=disp_str(field)) for field in fields),
        )
    button = ipw.Button(description="Save", button_style='warning')
    console = ipw.Output()
    
    rows = [probe_row(p) for p in probes]
    widget = ipw.VBox([*rows, button, console])
    
    def save(b):
        d = {}
        for letter, row in zip(probes, rows):
            p = d.get(probe(letter), {})
            for widget in row.children:
                v = widget.value
                if v in (None, False, ''):
                    continue
                if isinstance(widget, ipw.Text):
                    p['Notes'] = widget.value
                if isinstance(widget, ipw.Checkbox):
                    p[save_str(widget.description)] = widget.value
            if p:
                d[probe(letter)] = p  
        
        session.platform_json.InsertionNotes = d 
        with console:
            print('Updated notes')
        button.button_style = 'success'
        
    button.on_click(save)
    return IPython.display.display(widget)

insertion_notes_widget(session)

In [ ]:
import ipywidgets as ipw
import IPython.display
import np_session
import re

def probe_notes_widget(session: np_session.Session):
    
    probes = 'ABCDEF'
    probe = lambda _: f'Probe{_}'
    
    entry = lambda: session.platform_json.InsertionNotes
    
    get_notes = lambda _: entry().get(probe(_), {}).get('Notes', '')
    set_notes = lambda _, value: entry().setdefault(probe(_), {}).update({'Notes': value} if value else {})
    
    def disp_str(s): # split PascalCase fieldname into 'Title case' words
        matches = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', s)
        return ' '.join([m.group(0) for m in matches]).lower().capitalize()
    
    save_str = lambda s: ''.join([_.capitalize() for _ in s.split(' ')])
    
    fields = (
        "FailedToInsert",
        # "ProbeLocationChanged",
        # "ProbeBendingOnSurface",
        # "ProbeBendingElsewhere",
    )
    # "NumAgarInsertions",
    
    get_field = lambda _, field: entry().get(probe(_), {}).get(field, None)
    set_field = lambda _, field, value: entry().setdefault(probe(_), {}).update({field: value} if value in (True, False) else {})
    
    row = lambda *args: ipw.HBox([*args])
    probe_row = lambda p: row(
        ipw.Text(value=get_notes(p), placeholder='Insertion notes', description=disp_str(probe(p).strip('Probe ')), layout=ipw.Layout(width='auto', min_width='400px')),
        *(ipw.Checkbox(value=get_field(p, field), description=disp_str(field)) for field in fields),
        )
    button = ipw.Button(description="Save", button_style='warning')
    console = ipw.Output()
    
    rows = [probe_row(p) for p in probes]
    widget = ipw.VBox([*rows, button, console])
    
    def save(b):
        d = {}
        for probe, row in zip(probes, rows):
            for widget in row.children:
                if not widget.value:
                    continue
                if isinstance(widget, ipw.Text):
                    set_notes(probe, widget.value)
                if isinstance(widget, ipw.Checkbox):
                    set_field(probe, save_str(widget.description), widget.value)
        with console:
            print(entry())
        button.button_style = 'success'
        
    button.on_click(save)
    return IPython.display.display(widget)

probe_notes_widget(session)

In [ ]:
mc = np_session.Session('1252811121_661730_20230309').platform_json.manipulator_coordinates
session = np_session.Session('1246096278_366122_20230209')
mc.update({'pre_experiment_surface_image': {_ : {'x': 1, 'y': 2, 'z': 3000} for _ in 'ABCDEF'}})
mc.update({'post_experiment_surface_image': {_ : {'x': 1, 'y': 2, 'z': 6000.5} for _ in 'ABCDEF'}})
mc.update({'post_insertion_surface_image': {_ : {'x': 1, 'y': 2, 'z': 1000.2} for _ in 'ABCDEF'}})
mc.update({'pre_insertion_surface_image': {_ : {'x': 1, 'y': 2, 'z': 3} for _ in 'ABCDEF'}})
mc.update({'post_experiment_brain_surface_image': {_ : {'x': 1, 'y': 2, 'z': 3000} for _ in 'ABCDEF'}})
session.platform_json.manipulator_coordinates = mc

In [ ]:
session.platform_json.manipulator_coordinates

In [ ]:
probes = 'ABCDEF'
dict.fromkeys(probes, dict(x=0, y=0, z=0))

In [ ]:
np_workflows.print_countdown_timer(minutes=1)

In [ ]:
import ipywidgets as ipw
import IPython.display
import np_session
import re

def probe_depth_widget(session: np_session.Session):
    
    probes = 'ABCDEF'
    
    coords = lambda: session.platform_json.manipulator_coordinates
    
    if not coords():
        logger.warning("No photodocs have been captured yet.")
    
    probe_coords = lambda img: coords().get(img, dict.fromkeys(probes, dict(x=None, y=None, z=None)))
    field_str = lambda s: '_'.join(s.split(' ')).lower() + '_surface_image' if s else ''
    
    selection = ipw.ToggleButtons(
    options=[' '.join(_.strip('_surface_image').split('_')).capitalize() for _ in coords().keys()],
    description='Probe depth',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=[field_str(_) for _ in coords().keys()],
    )
    
    def update(_):
        for probe in probes:
            depth = probe_coords(field_str(selection.value))[probe]["z"]
            textbox[probe].value = f'{depth:6.1f}' if depth is not None else ''
            
    textbox = {
        probe: ipw.Text(
        value='', description=probe, 
        layout=ipw.Layout(max_width='150px'),)
        for probe in probes
    }
    selection.observe(update, 'value')
    update(None)
    widget = ipw.VBox([selection, ipw.HBox([*textbox.values()])])
    return IPython.display.display(widget)

# session.platform_json.manipulator_coordinates = {}
probe_depth_widget(session)

In [ ]:
session.platform_json.manipulator_coordinates

In [ ]:
import ipywidgets as ipw
import IPython.display
import np_session

def probe_notes_widget(session: np_session.Session):
    probes = 'ABCDEF'
    probe = lambda _: f'Probe{_}'
    
    entry = lambda: session.platform_json.InsertionNotes
    
    get_notes = lambda _: entry().get(probe(_), {}).get('Notes', '')
    set_notes = lambda _, value: entry().setdefault(probe(_), {}).update({'Notes': value} if value else {})
    
    fields = (
        "ProbeLocationChanged",
        "ProbeBendingOnSurface",
        "NumAgarInsertions",
        "FailedToInsert",
        "ProbeBendingElsewhere",
    )
    get_field = lambda _, field: entry().get(probe(_), {}).get(field, None)
    set_field = lambda _, field, value: entry().setdefault(probe(_), {}).update({field: value} if value in (True, False) else {})
    
    row = lambda *args: ipw.HBox([*args])
    probe_row = lambda p: row(
        ipw.Text(value=get_notes(p), placeholder='Notes', description=probe(p), layout=ipw.Layout(width='auto')),
        *(ipw.Checkbox(value=get_field(p, field), description=field) for field in fields),
        )
    button = ipw.Button(description="Save", button_style='warning')
    
    widget = ipw.VBox([*map(probe_row, probes), button])
    IPython.display.display(widget)

probe_notes_widget(session)

In [ ]:
np_workflows.di_widget(platform_json)

In [ ]:
np_workflows.wheel_height_widget(platform_json)

In [ ]:
np_workflows.pre_stim_check_widget()

In [ ]:
experiment = np_workflows.Ephys(session.mouse, session.user)
experiment.services

In [ ]:
np_services.Cam3d.data_files

In [ ]:
np_workflows.photodoc('sdf')

In [ ]:
np_services.NewScaleCoordinateRecorder.read()

In [ ]:
print(session.npexp_path)
np_services.NewScaleCoordinateRecorder.log_root = session.npexp_path
np_services.NewScaleCoordinateRecorder.log_name = session.platform_json.path.name
np_services.NewScaleCoordinateRecorder.label = 'test3'
# np_services.NewScaleCoordinateRecorder.initialize()
print(f'Current newscale log file {np_services.NewScaleCoordinateRecorder.get_current_log()}')
np_services.NewScaleCoordinateRecorder.start()
print(session.platform_json.path)
IPython.display.JSON(session.platform_json.path)

In [ ]:
import datetime
import np_config
np_config.normalize_time(datetime.datetime.strptime('2023/03/06 14:19:45.992', '%Y/%m/%d %H:%M:%S.%f'))

In [ ]:
np_workflows.check_hardware_widget()

In [ ]:
np_workflows.check_mouse_widget()

In [ ]:
np_workflows.check_openephys_widget()

In [ ]:
np_workflows.elapsed_time_widget()

In [ ]:
np_workflows.mtrain_widget(mouse)

In [ ]:
np_workflows.photodoc_widget('test_image')

In [ ]:
np_workflows.isi_widget(366122, colormap=False)

In [ ]:
np_workflows.isi_widget(366122, colormap=True)

In [ ]:
np_workflows.isi_widget(36612299999) # not in lims

In [ ]:
np_workflows.isi_widget(636766) # no ISI map

In [ ]:
np_workflows.print_countdown_timer(seconds=60)

In [ ]:
import tempfile
np_workflows.probe_targeting_widget(tempfile.tempdir)

In [ ]:
np_workflows.finishing_checks_widget()